In [ ]:
# Exercise Seven Start



# Exercise Seven Starter: Textual Analysis

In this exercise, I will not be providing the subheadings. Work from our code examples and the textbooks to construct a well-documented notebook that provides a model for initial textual analysis of a multi-document corpus.

Your workflow should:

- Import at least three documents you would like to compare (from text files, or using another format for a challenge.)
- Preprocess the text and create a tokenized corpus from the text of the imported documents.
- Create a document term matrix to enable comparative textual analysis across the full set of documents
- Chart at least one comparison between the documents, using word frequency to map the text
- Calculate the Euclidean distance between the documents, using two key words as the point of comparison

As a bonus challenge, consider trying one of the other types of distance modeling described in the text.

## Import Documents

The documents selected are:

- a
- b
- c
- d